<a href="https://colab.research.google.com/github/Sanyam8055/EIP-4.0-/blob/master/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ResNet50 with Cyclic LR 

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/My Drive/Colab Notebooks/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls
%tensorflow_version 1.x

import cv2
import json
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16,InceptionV3, ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam,SGD,RMSprop
from keras.preprocessing.image import ImageDataGenerator
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation
    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.25)
train_df.shape, val_df.shape
train_df.head()
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=512)
valid_gen = PersonDataGenerator(train_df, batch_size=256, shuffle=True)
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive/  hvc_annotations.csv  resized/  sample_data/


Using TensorFlow backend.


{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
backbone = ResNet50(
    weights=None, 
    include_top=True, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output



def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)


 #model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)
model.summary()

filepath="/content/drive/My Drive/Colab Notebooks/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1,save_weights_only=False, save_best_only=True)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
from keras.models import load_model
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights.best.hdf5')

In [0]:
from keras.callbacks import *
#Cyclic Learning Rate
class CyclicLR(Callback):
  

    def __init__(self, base_lr=0.00005, max_lr=0.0001, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())


In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=False,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=0,

    )
)
clr_triangular = CyclicLR(mode='triangular2')
model.fit_generator(  
    steps_per_epoch=100000 // 64,  validation_steps=10000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=20,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)  

Epoch 1/20
1562/1562 [==============================] - 1085s 695ms/step - loss: 6.5103 - gender_output_loss: 0.1735 - image_quality_output_loss: 0.9735 - age_output_loss: 1.2987 - weight_output_loss: 0.8997 - bag_output_loss: 0.8499 - footwear_output_loss: 0.5063 - pose_output_loss: 0.9195 - emotion_output_loss: 0.8890 - gender_output_acc: 0.9164 - image_quality_output_acc: 0.5555 - age_output_acc: 0.4428 - weight_output_acc: 0.6666 - bag_output_acc: 0.6044 - footwear_output_acc: 0.7708 - pose_output_acc: 0.6162 - emotion_output_acc: 0.7091 - val_loss: 6.1076 - val_gender_output_loss: 0.0281 - val_image_quality_output_loss: 0.9718 - val_age_output_loss: 1.2454 - val_weight_output_loss: 0.8689 - val_bag_output_loss: 0.8306 - val_footwear_output_loss: 0.3644 - val_pose_output_loss: 0.9165 - val_emotion_output_loss: 0.8819 - val_gender_output_acc: 0.9977 - val_image_quality_output_acc: 0.5557 - val_age_output_acc: 0.4615 - val_weight_output_acc: 0.6718 - val_bag_output_acc: 0.6373 - val_

In [0]:

backbone = ResNet50(
    weights=None, 
    include_top=True, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output



def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)


opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)

 #model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)
model.summary()
 # checkpoint
filepath="/content/drive/My Drive/Colab Notebooks/weights2.best.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
callbacks_list = [checkpoint]     
  
 














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256     

In [0]:
from keras.models import load_model
from keras import backend as k
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights.best.hdf5')
k.set_value(model.optimizer.lr,  0.0005)
k.set_value(model.optimizer.momentum,  0.9)

In [0]:
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,callbacks=[checkpoint]
)  

Epoch 1/25
312/312 [==============================] - 107s 343ms/step - loss: 6.8417 - gender_output_loss: 0.3108 - image_quality_output_loss: 0.9733 - age_output_loss: 1.3265 - weight_output_loss: 0.9246 - bag_output_loss: 0.8574 - footwear_output_loss: 0.6370 - pose_output_loss: 0.9215 - emotion_output_loss: 0.8907 - gender_output_acc: 0.8668 - image_quality_output_acc: 0.5557 - age_output_acc: 0.4322 - weight_output_acc: 0.6583 - bag_output_acc: 0.6060 - footwear_output_acc: 0.7387 - pose_output_acc: 0.6162 - emotion_output_acc: 0.7099 - val_loss: 6.5136 - val_gender_output_loss: 0.1622 - val_image_quality_output_loss: 0.9708 - val_age_output_loss: 1.2987 - val_weight_output_loss: 0.9111 - val_bag_output_loss: 0.8363 - val_footwear_output_loss: 0.5284 - val_pose_output_loss: 0.9230 - val_emotion_output_loss: 0.8832 - val_gender_output_acc: 0.9482 - val_image_quality_output_acc: 0.5578 - val_age_output_acc: 0.4538 - val_weight_output_acc: 0.6547 - val_bag_output_acc: 0.6178 - val_foo

In [0]:
from keras.models import load_model
from keras import backend as k
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights1.best.hdf5')
k.set_value(model.optimizer.lr,  0.0005)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,callbacks=[checkpoint]
) 

Epoch 1/25
312/312 [==============================] - 91s 293ms/step - loss: 6.3963 - gender_output_loss: 0.1624 - image_quality_output_loss: 0.9726 - age_output_loss: 1.2390 - weight_output_loss: 0.8690 - bag_output_loss: 0.8530 - footwear_output_loss: 0.5062 - pose_output_loss: 0.9149 - emotion_output_loss: 0.8792 - gender_output_acc: 0.9192 - image_quality_output_acc: 0.5570 - age_output_acc: 0.4714 - weight_output_acc: 0.6841 - bag_output_acc: 0.6075 - footwear_output_acc: 0.7817 - pose_output_acc: 0.6152 - emotion_output_acc: 0.7096 - val_loss: 6.0619 - val_gender_output_loss: 0.0243 - val_image_quality_output_loss: 0.9694 - val_age_output_loss: 1.1923 - val_weight_output_loss: 0.8546 - val_bag_output_loss: 0.8414 - val_footwear_output_loss: 0.3743 - val_pose_output_loss: 0.9153 - val_emotion_output_loss: 0.8904 - val_gender_output_acc: 0.9958 - val_image_quality_output_acc: 0.5565 - val_age_output_acc: 0.4922 - val_weight_output_acc: 0.7050 - val_bag_output_acc: 0.6140 - val_foot

In [0]:
from keras.models import load_model
from keras import backend as k
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights1.best.hdf5')
k.set_value(model.optimizer.lr,  0.0005)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,callbacks=[checkpoint]
) 

Epoch 1/25
312/312 [==============================] - 90s 288ms/step - loss: 6.2633 - gender_output_loss: 0.1215 - image_quality_output_loss: 0.9711 - age_output_loss: 1.1994 - weight_output_loss: 0.8467 - bag_output_loss: 0.8535 - footwear_output_loss: 0.4749 - pose_output_loss: 0.9140 - emotion_output_loss: 0.8821 - gender_output_acc: 0.9550 - image_quality_output_acc: 0.5564 - age_output_acc: 0.4902 - weight_output_acc: 0.6961 - bag_output_acc: 0.6207 - footwear_output_acc: 0.7918 - pose_output_acc: 0.6164 - emotion_output_acc: 0.7086 - val_loss: 5.8682 - val_gender_output_loss: 0.0151 - val_image_quality_output_loss: 0.9665 - val_age_output_loss: 1.1086 - val_weight_output_loss: 0.7950 - val_bag_output_loss: 0.8365 - val_footwear_output_loss: 0.3642 - val_pose_output_loss: 0.9121 - val_emotion_output_loss: 0.8702 - val_gender_output_acc: 0.9981 - val_image_quality_output_acc: 0.5599 - val_age_output_acc: 0.5218 - val_weight_output_acc: 0.7364 - val_bag_output_acc: 0.6492 - val_foot

In [0]:
from keras.models import load_model
from keras import backend as k
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights1.best.hdf5')
k.set_value(model.optimizer.lr,  0.0005)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,callbacks=[checkpoint]
) 

Epoch 1/25
312/312 [==============================] - 90s 288ms/step - loss: 6.1742 - gender_output_loss: 0.1049 - image_quality_output_loss: 0.9662 - age_output_loss: 1.1654 - weight_output_loss: 0.8439 - bag_output_loss: 0.8422 - footwear_output_loss: 0.4582 - pose_output_loss: 0.9143 - emotion_output_loss: 0.8790 - gender_output_acc: 0.9564 - image_quality_output_acc: 0.5643 - age_output_acc: 0.5103 - weight_output_acc: 0.6917 - bag_output_acc: 0.6288 - footwear_output_acc: 0.7954 - pose_output_acc: 0.6154 - emotion_output_acc: 0.7091 - val_loss: 5.7828 - val_gender_output_loss: 0.0072 - val_image_quality_output_loss: 0.9664 - val_age_output_loss: 1.0489 - val_weight_output_loss: 0.7842 - val_bag_output_loss: 0.8225 - val_footwear_output_loss: 0.3680 - val_pose_output_loss: 0.9065 - val_emotion_output_loss: 0.8791 - val_gender_output_acc: 0.9982 - val_image_quality_output_acc: 0.5541 - val_age_output_acc: 0.5415 - val_weight_output_acc: 0.7104 - val_bag_output_acc: 0.6502 - val_foot

In [0]:
from keras.models import load_model
from keras import backend as k
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights1.best.hdf5')
k.set_value(model.optimizer.lr,  0.0005)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,callbacks=[checkpoint]
) 

Epoch 1/25
312/312 [==============================] - 90s 288ms/step - loss: 6.0646 - gender_output_loss: 0.0887 - image_quality_output_loss: 0.9665 - age_output_loss: 1.1044 - weight_output_loss: 0.8157 - bag_output_loss: 0.8368 - footwear_output_loss: 0.4531 - pose_output_loss: 0.9154 - emotion_output_loss: 0.8841 - gender_output_acc: 0.9717 - image_quality_output_acc: 0.5617 - age_output_acc: 0.5447 - weight_output_acc: 0.7029 - bag_output_acc: 0.6320 - footwear_output_acc: 0.8041 - pose_output_acc: 0.6160 - emotion_output_acc: 0.7084 - val_loss: 5.6828 - val_gender_output_loss: 0.0099 - val_image_quality_output_loss: 0.9611 - val_age_output_loss: 0.9863 - val_weight_output_loss: 0.7580 - val_bag_output_loss: 0.8213 - val_footwear_output_loss: 0.3560 - val_pose_output_loss: 0.9143 - val_emotion_output_loss: 0.8760 - val_gender_output_acc: 0.9979 - val_image_quality_output_acc: 0.5655 - val_age_output_acc: 0.6027 - val_weight_output_acc: 0.7491 - val_bag_output_acc: 0.6772 - val_foot

In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=False,
        horizontal_flip=False,
        vertical_flip=True,
        shear_range=0.2,
        zoom_range=0.8,
        rotation_range=45,

    )
)
from keras.models import load_model
from keras import backend as k
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights1.best.hdf5')
k.set_value(model.optimizer.lr,  0.0001)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,callbacks=[checkpoint]
) 

Epoch 1/25
312/312 [==============================] - 90s 288ms/step - loss: 6.0018 - gender_output_loss: 0.0824 - image_quality_output_loss: 0.9675 - age_output_loss: 1.0573 - weight_output_loss: 0.8087 - bag_output_loss: 0.8405 - footwear_output_loss: 0.4423 - pose_output_loss: 0.9187 - emotion_output_loss: 0.8843 - gender_output_acc: 0.9716 - image_quality_output_acc: 0.5602 - age_output_acc: 0.5631 - weight_output_acc: 0.7082 - bag_output_acc: 0.6259 - footwear_output_acc: 0.8069 - pose_output_acc: 0.6153 - emotion_output_acc: 0.7092 - val_loss: 5.5871 - val_gender_output_loss: 0.0037 - val_image_quality_output_loss: 0.9660 - val_age_output_loss: 0.9200 - val_weight_output_loss: 0.7369 - val_bag_output_loss: 0.8170 - val_footwear_output_loss: 0.3466 - val_pose_output_loss: 0.9146 - val_emotion_output_loss: 0.8823 - val_gender_output_acc: 0.9994 - val_image_quality_output_acc: 0.5582 - val_age_output_acc: 0.6355 - val_weight_output_acc: 0.7581 - val_bag_output_acc: 0.6746 - val_foot

In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=False,
        vertical_flip=True,
        shear_range=0.2,
        zoom_range=[0.5,1.0],
        rotation_range=30,
        brightness_range=[0.2,1.0]

    )
)
from keras.models import load_model
from keras import backend as k
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights1.best.hdf5')
k.set_value(model.optimizer.lr,  0.0005)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=100,
    verbose=1,callbacks=[checkpoint]
) 

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
312/312 [==============================] - 235s 752ms/step - loss: 6.3827 - gender_output_loss: 0.2064 - image_quality_output_loss: 0.9703 - age_output_loss: 1.1868 - weight_output_loss: 0.8571 - bag_output_loss: 0.8390 - footwear_output_loss: 0.5258 - pose_output_loss: 0.9154 - emotion_output_loss: 0.8819 - gender_output_acc: 0.9382 - image_quality_output_acc: 0.5567 - age_output_acc: 0.4990 - weight_output_acc: 0.6819 - bag_output_acc: 0.6315 - footwear_output_acc: 0.7864 - pose_output_acc: 0.6191 - emotion_output_acc: 0.7124 - val_loss: 5.9216 - val_gender_output_loss: 0.0736 - val_image_quality_output_loss: 0.9732 - val_age_output_loss: 1.0392 - val_weight_output_loss: 0.8083 - val_bag_output_loss: 0.8189 - val_footwear_output_loss: 0.4202 - val_pose_output_loss: 0.9166 - val_emotion_output_loss: 0.8717 - val_gender_output_acc: 0.9810 - val_image_quality_output_acc: 0.5488 - 

In [0]:
filepath="/content/drive/My Drive/Colab Notebooks/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1,save_weights_only=False, save_best_only=True)
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=False,
        vertical_flip=False,
        shear_range=0.4,
        zoom_range=[0.5,1.0],
        rotation_range=30,
        brightness_range=[0.2,1.0],
        width_shift_range=0.2,
	      height_shift_range=0.2,
	fill_mode="nearest")
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001)
from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.0004)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint]
) 


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/100
312/312 [==============================] - 216s 692ms/step - loss: 5.6301 - gender_output_loss: 0.1034 - image_quality_output_loss: 0.9605 - age_output_loss: 0.8725 - weight_output_loss: 0.7551 - bag_output_loss: 0.7652 - footwear_output_loss: 0.3890 - pose_output_loss: 0.9044 - emotion_output_loss: 0.8800 - gender_output_acc: 0.9606 - image_quality_output_acc: 0.5612 - age_output_acc: 0.6298 - weight_output_acc: 0.7214 - bag_output_acc: 0.6732 - footwear_output_acc: 0.8300 - pose_output_acc: 0.6199 - emotion_output_acc: 0.7107 - val_loss: 5.1926 - val_gender_output_loss: 0.0034 - val_image_quality_output_loss: 0.9560 - val_age_output_loss: 0.7295 - val_weight_output_loss: 0.7093 - val_bag_output_loss: 0.7285 - val_footwear_output_loss: 0.2927 - val_pose_output_loss: 0.8990 - val_emotion_output_loss: 0.8742 - val_gender_output_acc: 0.9994 - val_image_quality_output_acc: 0.5593 - val_age_output_acc: 0.7359 - val_weight_output_acc: 0.7675 - val_bag_output_acc: 0.7087 - val_fo

In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights.72-4.75.hdf5')
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=False,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.2,
        zoom_range=[0.6,1.0],
        rotation_range=60,
        brightness_range=[0.4,1.0],
        width_shift_range=0.2,
	      height_shift_range=0.2,
        )     
    )

opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0000001)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.000005)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=100,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
312/312 [==============================] - 107s 342ms/step - loss: 5.7616 - gender_output_loss: 0.1376 - image_quality_output_loss: 0.9641 - age_output_loss: 0.9550 - weight_output_loss: 0.7630 - bag_output_loss: 0.7471 - footwear_output_loss: 0.4148 - pose_output_loss: 0.9081 - emotion_output_loss: 0.8719 - gender_output_acc: 0.9518 - image_quality_output_acc: 0.5616 - age_output_acc: 0.6234 - weight_output_acc: 0.7159 - bag_output_acc: 0.6855 - footwear_output_acc: 0.8377 - pose_output_acc: 0.6200 - emotion_output_acc: 0.7121 - val_loss: 5.1824 - val_gender_output_loss: 0.0367 - val_image_quality_output_loss: 0.9552 - val_age_output_loss: 0.7614 - val_weight_output_loss: 0.6932 - val_bag_output_loss: 0.6723 - val_footwear_output_loss: 0.3026 - val_pose_output_loss: 0.9021 - val_emotion_output_loss: 0.8590 - val_gender_output_acc: 0.9940 - val_image_quality_output_acc: 0.5630 - 

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=False,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.4,
        zoom_range=[0.3,1.0],
        
        brightness_range=[0.2,1.0],
        width_shift_range=0.2,
	      height_shift_range=0.2,
	      fill_mode="nearest",
        zca_epsilon=1e-06,
        channel_shift_range=0.5,)     
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.0003)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=100,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 

Epoch 1/100
312/312 [==============================] - 91s 291ms/step - loss: 5.6266 - gender_output_loss: 0.1288 - image_quality_output_loss: 0.9614 - age_output_loss: 0.8772 - weight_output_loss: 0.7541 - bag_output_loss: 0.7397 - footwear_output_loss: 0.3856 - pose_output_loss: 0.9068 - emotion_output_loss: 0.8729 - gender_output_acc: 0.9512 - image_quality_output_acc: 0.5613 - age_output_acc: 0.6426 - weight_output_acc: 0.7158 - bag_output_acc: 0.6893 - footwear_output_acc: 0.8393 - pose_output_acc: 0.6208 - emotion_output_acc: 0.7115 - val_loss: 5.0162 - val_gender_output_loss: 0.0236 - val_image_quality_output_loss: 0.9527 - val_age_output_loss: 0.6715 - val_weight_output_loss: 0.6947 - val_bag_output_loss: 0.6514 - val_footwear_output_loss: 0.2653 - val_pose_output_loss: 0.8993 - val_emotion_output_loss: 0.8579 - val_gender_output_acc: 0.9971 - val_image_quality_output_acc: 0.5662 - val_age_output_acc: 0.7511 - val_weight_output_acc: 0.7308 - val_bag_output_acc: 0.7484 - val_foo

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.4,
        zoom_range=[0.3,1.0],
        rotation_range=50,
        brightness_range=[0.2,1.0],
        width_shift_range=0.2,
	      height_shift_range=0.2,
	      fill_mode="nearest",
        channel_shift_range=0.5,)     
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000009)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.00005)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
312/312 [==============================] - 92s 295ms/step - loss: 5.2202 - gender_output_loss: 0.1088 - image_quality_output_loss: 0.9621 - age_output_loss: 0.7405 - weight_output_loss: 0.6814 - bag_output_loss: 0.6555 - footwear_output_loss: 0.3164 - pose_output_loss: 0.8952 - emotion_output_loss: 0.8604 - gender_output_acc: 0.9548 - image_quality_output_acc: 0.5588 - age_output_acc: 0.6925 - weight_output_acc: 0.7421 - bag_output_acc: 0.7355 - footwear_output_acc: 0.8729 - pose_output_acc: 0.6328 - emotion_output_acc: 0.7116 - val_loss: 4.6138 - val_gender_output_loss: 0.0130 - val_image_quality_output_loss: 0.9535 - val_age_output_loss: 0.5391 - val_weight_output_loss: 0.6016 - val_bag_output_loss: 0.5546 - val_footwear_output_loss: 0.2171 - val_pose_output_loss: 0.8924 - val_emotion_output_loss: 0.8425 - val_gender_output_acc: 0.9997 - val_image_quality_output_acc: 0.5677 - val_age_output_acc: 0.8102 - val_weight_output_acc: 0.7887 - val_bag_output_acc: 0.7986 - val_foot

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.6,
        zoom_range=[0.2,1.0],
        rotation_range=30,
        brightness_range=[0.4,1.0],
        width_shift_range=0.2,
	      height_shift_range=0.2,
	fill_mode="nearest")
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.00004)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
312/312 [==============================] - 93s 297ms/step - loss: 5.1851 - gender_output_loss: 0.1076 - image_quality_output_loss: 0.9632 - age_output_loss: 0.7316 - weight_output_loss: 0.6768 - bag_output_loss: 0.6426 - footwear_output_loss: 0.3080 - pose_output_loss: 0.8948 - emotion_output_loss: 0.8604 - gender_output_acc: 0.9571 - image_quality_output_acc: 0.5609 - age_output_acc: 0.6952 - weight_output_acc: 0.7411 - bag_output_acc: 0.7450 - footwear_output_acc: 0.8764 - pose_output_acc: 0.6310 - emotion_output_acc: 0.7124 - val_loss: 4.5783 - val_gender_output_loss: 0.0126 - val_image_quality_output_loss: 0.9562 - val_age_output_loss: 0.5395 - val_weight_output_loss: 0.5944 - val_bag_output_loss: 0.5506 - val_footwear_output_loss: 0.2077 - val_pose_output_loss: 0.8761 - val_emotion_output_loss: 0.8412 - val_gender_output_acc: 1.0000 - val_image_quality_output_acc: 0.5644 - val_age_output_acc: 0.8141 - val_weight_output_acc: 0.7848 - val_bag_output_acc: 0.8033 - val_foot

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.4,
	fill_mode="nearest")
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.00000001)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.000004)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
625/625 [==============================] - 165s 263ms/step - loss: 5.1773 - gender_output_loss: 0.1115 - image_quality_output_loss: 0.9599 - age_output_loss: 0.7238 - weight_output_loss: 0.6753 - bag_output_loss: 0.6437 - footwear_output_loss: 0.3111 - pose_output_loss: 0.8945 - emotion_output_loss: 0.8575 - gender_output_acc: 0.9566 - image_quality_output_acc: 0.5631 - age_output_acc: 0.6986 - weight_output_acc: 0.7449 - bag_output_acc: 0.7434 - footwear_output_acc: 0.8722 - pose_output_acc: 0.6331 - emotion_output_acc: 0.7118 - val_loss: 4.5586 - val_gender_output_loss: 0.0129 - val_image_quality_output_loss: 0.9474 - val_age_output_loss: 0.5268 - val_weight_output_loss: 0.5873 - val_bag_output_loss: 0.5409 - val_footwear_output_loss: 0.2111 - val_pose_output_loss: 0.8848 - val_emotion_output_loss: 0.8472 - val_gender_output_acc: 1.0000 - val_image_quality_output_acc: 0.5713 - val_age_output_acc: 0.8177 - val_weight_output_acc: 0.7903 - val_bag_output_acc: 0.8100 - val_foo

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.4,
	fill_mode="nearest")
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.00000001)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.000004)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.4,
	fill_mode="nearest")
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.00000001)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.000004)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        shear_range=0.4,
	fill_mode="nearest")
    )


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.00000001)

from keras.models import load_model
from keras import backend as k
k.set_value(model.optimizer.lr,  0.000004)
k.set_value(model.optimizer.momentum,  0.9)
model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,reduce_lr]
) 